In [1]:
import datetime
import pandas as pd
from smart_open import open
import pymysql.cursors
# import rds_config
import s3_file_operations as s3_ops

We save our RDS credentials via the following variables. please note this is not a safe way to access credentials, its advisable to use environmental variables instead

In [2]:
rds_host = "rick-and-morty-****************8.eu-west-1.rds.amazonaws.com" # Replace with your RDS hostname
rds_username = "admin"
rds_user_pwd = "******"  # Replace with your password
rds_db_name = "rick_and_morty"
bucket_name = "de-masterclass"   # Replace with your s3 Bucket name

Reading one of our Transformed Datasets from s3

In [4]:
# Reading Character.csv file from Transformed folder in s3
latest_records = s3_ops.read_csv_from_s3(bucket_name, "Rick&Morty/Transformed/Character.csv")
latest_records = latest_records.fillna('NULL')

In [5]:
# Defining our SQL create table query 

create_sql_script = """
        CREATE TABLE Character_Table (
            id INT NOT NULL PRIMARY KEY,
            name VARCHAR(255),
            status VARCHAR(255),
            species VARCHAR(255),
            type VARCHAR(255),
            gender VARCHAR(255),
            origin VARCHAR(255),
            location VARCHAR(255),
            image VARCHAR(255),
            episode VARCHAR(255),
            url VARCHAR(255),
            created VARCHAR(255)) 
        ENGINE=INNODB;
    """

Defining our connection to our remote database on AWS, creating our Destination Table and finally inserting the just transformed records to the database. 

In [6]:
try:
    conn = pymysql.connect(host=rds_host,
                            user=rds_username,
                            password=rds_user_pwd,
                            port=3306,
                            database=rds_db_name,
                            cursorclass=pymysql.cursors.DictCursor)
    cursor = conn.cursor()

    cursor.execute("DROP TABLE IF EXISTS Characters")
    cursor.execute(create_sql_script)
    
    # define the list of columns to insert data into
    column_names = list(latest_records.columns)
    
    for i, row in latest_records.iterrows():
        # create a string with the correct number of placeholders for the data
        placeholders = ','.join(['%s']*len(column_names))
        # create the SQL insert statement with the specified column names and placeholders
        sql_insert = f"INSERT INTO Characters ({','.join(column_names)}) VALUES ({placeholders});"
        # extract the data for the specified columns from the row
        data = tuple(row[column] for column in column_names)
        # execute the insert statement with the data
        cursor.execute(sql_insert, data)
        conn.commit()
        
    print(f"finished insertion of {latest_records.shape[0]} Character records to RDS...")
        

except Exception as e:
    print("Exception: ", e)

Exception:  (2003, "Can't connect to MySQL server on 'rick-and-morty-db.ctk2ewoqitft.eu-west-1.rds.amazonaws.com' (timed out)")
